In [1]:
import time
import threading
from pynput import keyboard
from IPython.display import clear_output
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import cv2
import imutils
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.collections import LineCollection
import matplotlib.patches as patches
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import pandas as pd
import imageio
from helpers.activate_software import *
from helpers.movenet_helpers import *
from tqdm.notebook import tqdm_notebook
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
import pickle
i = 0
prob = 0

def mytextfunction(x):
    print("You have typed.... " + str(x) + " characters.")
    return x

def on_press(key):
    global i
    i += 1
    clear_output(wait=True)
    print(f"You have typed {i} characters", flush = True)
    int_progress = widgets.IntProgress(
    value=i,
    min=0,
    max=100,
    step=1,
    description='Characters :',
    bar_style='', 
    orientation='horizontal'
    )
    display(int_progress)
    
def on_release(key):
    if(i > 99):
        print("You have worked too hard.  Stop working now and go Yoga!!!")
        display(button2, output2)
        return False
    if(key == keyboard.Key.esc):
        return False    

def on_button_clicked(b):
    i = 0
    with output:
        print("Button clicked. Start counting")
    with keyboard.Listener(on_press = on_press, on_release = on_release) as listener:
        listener.join()
def activate_software():
    import cv2
    import imutils
    import matplotlib.pyplot as plt
    from matplotlib.image import imread
    %matplotlib inline

    #cv2.namedWindow("RSI Prevention using Yoga Pose")
    vc = cv2.VideoCapture(0)

    if vc.isOpened(): # try to get the first frame
        rval, frame = vc.read()
    else:
        rval = False

    while rval:
        rval, frame = vc.read()
        frame = imutils.resize(frame, width=600) 
        cv2.imwrite("test01.jpg", frame)
        image_path = r'C:\project\YogaPoseEstimationPython\YogaPoseEstimation\yoga\test01.jpg'
        keypoints_with_score = get_keypoint(image_path, input_size)
        keypoints = keypoints_with_score[0][0]        
        # Visualize the predictions with image.
        display_image = tf.expand_dims(frame, axis=0)
        display_image = tf.cast(tf.image.resize_with_pad(
            display_image, 450, 600), dtype=tf.int32)
        output_overlay = draw_prediction_on_image(
            np.squeeze(display_image.numpy(), axis=0), keypoints_with_score)
        resized_overlay = cv2.resize(output_overlay, (0, 0), None, .5, .5)
        tree = cv2.imread('C:\project\Ref_Images\Tree.png')
        tree = cv2.resize(tree, (576, 432))
        stack = np.hstack((resized_overlay, tree))        
       
        predicted_class, prob = get_prediction(keypoints)
        if predicted_class == 'tree' :
            cv2.putText(stack, '{} with proability {:.2f}'.format(predicted_class, prob), (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)            
            
            goddess = cv2.imread('C:\project\Ref_Images\goddess.png')
            goddess = cv2.resize(goddess, (576, 432))
            stack = np.hstack((resized_overlay, goddess)) 
           # if predicted_class == 'goddess' :
           #     cv2.putText(stack, '{} with proability {:.2f}'.format(predicted_class, prob), (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)            
            
        else:
            cv2.putText(stack, 'No Pose Detected', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        cv2.imshow("RSI Prevention Using Yoga Pose", stack)
        key = cv2.waitKey(200)

        if key == 27: # exit on ESC
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            print(frame.shape)
            plt.imshow(frame)
            vc.release()
            #cv2.destroyWindow("RSI Prevention using Yoga Pose")
            break            
              
    
def on_button_clicked2(b2):
    with output2:
        print("Starting RSI prevention software...")
        activate_software()      

model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoint_with_scores = outputs['output_0'].numpy()
    return keypoint_with_scores

def get_keypoint(image_path, input_size):
    import tensorflow as tf
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    # Run model inference.
    keypoint_with_scores = movenet(input_image)
    
    return keypoint_with_scores

In [2]:
button = widgets.Button(description="Start Program")
output = widgets.Output()
display(button, output)

button.on_click(on_button_clicked)
button2 = widgets.Button(description="Starting software...")
output2 = widgets.Output()
button2.on_click(on_button_clicked2)

You have typed 101 characters


IntProgress(value=100, description='Characters :')

You have worked too hard.  Stop working now and go Yoga!!!


Button(description='Starting software...', style=ButtonStyle())

Output()

In [ ]:
#adsfadsfadsfadsfadsgawklhtkesajflsalfdjadlsjflasdjfldsafasdfadsfadsfadsfasdfasdfewfasdfasdfadfadsfadsfdasfsad